In [1]:
import sys
import json
import struct
from random import randint
from math import ceil
from pprint import pprint
import socket
import argparse

packet_storage = {}

JSON_PACKET = {'length': '00000000',
               'ID': '00000000',
               'FragFlag': '00000000',
               'Offset': '00000000'
               }

id_seed = randint(100, 999)
origin_packets = []
payload_key_size = 19  # hardcoding extra size of JSON_PACKET for 'payload' key

In [2]:
def pack_int_to_hex(data):
    def bytes_to_hex(byte_str):
        #return ''.join(["%02X" % ord(x) for x in byte_str]).strip()
        return byte_str.hex()
    return bytes_to_hex(struct.pack(">I", data))


def unpack_hex_to_int(data):
    return int(data, 16)


def size_of_json_packet(json_data):
    return sys.getsizeof(json.dumps(json_data)) + 20  # 20 = size of header


header_size = size_of_json_packet(json.dumps(JSON_PACKET))

In [3]:
def create_packet(payload, packet_id=0, offset=0, frag_flag=0):
    new_packet = dict(JSON_PACKET)
    new_packet['payload'] = payload

    new_packet['length'] = pack_int_to_hex(header_size + payload_key_size + len(payload))
    new_packet['Offset'] = pack_int_to_hex(offset)
    new_packet['FragFlag'] = pack_int_to_hex(frag_flag)
    new_packet['ID'] = pack_int_to_hex(packet_id)
    return new_packet

In [4]:
def get_x_bytes(byte_size, data, offset=0):
    ex_data, i = [], 0
    for x in data[offset:]:
        ex_data.append(x)

        new_packet = create_packet(''.join(ex_data))
        if size_of_json_packet(json.dumps(new_packet)) == byte_size:
            break
    return ''.join(ex_data), offset + i + 1

In [5]:
def create_original_packets(input_filename, packet_size):
    with open(input_filename) as myfile:
        sample_data = myfile.read()
    sample_data = json.dumps(sample_data)
    sample_data_length = len(sample_data)
    size_of_data_section = packet_size - (header_size + payload_key_size)
    number_of_packets = int(ceil(sample_data_length / float(size_of_data_section)))

    start_addr = 0
    for x in range(number_of_packets):
        payload, start_addr = get_x_bytes(packet_size, sample_data, start_addr)

        new_packet = create_packet(payload, packet_id=id_seed + x)
        start_addr += size_of_data_section

        origin_packets.append(new_packet)

    # print_packet(origin_packets)
    print("{} packets created with max {} bytes packet size to be sent ".format(len(origin_packets), packet_size))
    print("Header Size of each packet: {} bytes  ".format(header_size))
    for i, x in enumerate(origin_packets):
        print("Size of packet-{} is {} bytes".format(i + 1, size_of_json_packet(json.dumps(x))))
    return origin_packets

In [6]:
def create_packet_fragments(MTU, packet):
    size_of_data_section = MTU - header_size - payload_key_size
    input_data = packet['payload']
    data_length = len(packet['payload'])
    number_of_packet_fragments = int(ceil(data_length / float(size_of_data_section)))
    packet_fragments = []
    packet_id = unpack_hex_to_int(packet['ID'])

    start_addr = 0
    offset = 0
    for x in range(number_of_packet_fragments):

        if x != number_of_packet_fragments - 1:
            frag_flag = 1
        else:
            frag_flag = 0

        payload, start_addr = get_x_bytes(MTU, input_data, start_addr)
        new_packet = create_packet(payload, packet_id, offset, frag_flag)
        offset += size_of_data_section // 8
        start_addr += size_of_data_section

        packet_fragments.append(new_packet)
    # print_packet(packet_fragments)
    print("{} packet fragments created with max {} bytes packet size to be sent ".format(len(packet_fragments), MTU))
    for i, x in enumerate(packet_fragments):
        print("Size of fragment packet-{} is {} bytes".format(i + 1, size_of_json_packet(json.dumps(x))))
    return packet_fragments

In [7]:
def udp_sender(dest_ip, dest_port, data):
    # define socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # UDP
    print("\nSending all packets to {}:{}".format(dest_ip, dest_port))
    for x in range(0, len(data), 1024):
        sock.sendto(str.encode(data[x:x + 1024]), (dest_ip, dest_port))

In [8]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Creates packet fragments and send via UDP')
    parser.add_argument('-f', '--input_filename', help='Input filename', default='/home/cs/Documents/sample.txt')
    parser.add_argument('-ps', '--packet_size', help='Packet Size', type=int, default=5000)
    parser.add_argument('-mtu', '--MTU', help='MTU', type=int, default=3000)
    parser.add_argument('-ip', '--destIP', help='destination IP address', default='127.0.0.1')
    parser.add_argument('-port', '--destPort', help='destination port number', type=int, default=2001)

    args = parser.parse_args()

    orig_packets = create_original_packets(args.input_filename, args.packet_size)

    all_fragments = []
    for i, x in enumerate(orig_packets):
        print("\nFor original packet {}".format(i + 1))

        packet_fragments_list = create_packet_fragments(args.MTU, x)
        all_fragments += packet_fragments_list

    # printing all fragments to file for verification purpose
    with open("fragmented_packets.txt", 'w+') as myfile:
        pprint(all_fragments, myfile)

    udp_sender(args.destIP, args.destPort, str(json.dumps(all_fragments)))

1 packets created with max 5000 bytes packet size to be sent 
Header Size of each packet: 173 bytes  
Size of packet-1 is 717 bytes

For original packet 1
1 packet fragments created with max 3000 bytes packet size to be sent 
Size of fragment packet-1 is 717 bytes

Sending all packets to 127.0.0.1:2001
